In [1]:
# install chromedriver and selenium
!pip install chromedriver-py==101.0.4951.41
!pip install selenium
!pip install matplotlib
!pip install yaspin
!pip install unidecode
!pip install scrapetube
!pip install yt_dlp

In [25]:
!pwd

/data/notebooks/DATA-ISBAINE/Youtube


In [2]:
# importing necessary packages
from __future__ import unicode_literals
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from chromedriver_py import binary_path
import time
from selenium import webdriver
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
%matplotlib inline
from unidecode import unidecode
import time
from yaspin import yaspin
import itertools
import os
import scrapetube
from os.path import exists
from datetime import datetime
from datetime import date
import yt_dlp

# Don't Show Warning Messages

In [3]:
# creating a function that creates a chrome options object with given parameters
def add_driver_options(options):
    """
    This function sets driver options ( given as parameter)
    """
    chrome_options = Options()
    for opt in options:
        chrome_options.add_argument(opt)
    return chrome_options

# create function that initialize the driver and return it


def initialize_driver():
    """
    This function create an instace of the web driver with the options below:
    """
    driver_config = {
        "executable_path": binary_path,
        "options": [
            "--headless",
            "--no-sandbox",
            "--start-fullscreen",
            "--allow-insecure-localhost",
            "--disable-dev-shm-usage",
            
        ],
    }
    options = add_driver_options(driver_config["options"])
    options.add_experimental_option("prefs", {"profile.default_content_setting_values.cookies": 2})

    driver = webdriver.Chrome(
        executable_path=driver_config["executable_path"], options=options
    )
    return driver

In [4]:
# create a driver with initialize_driver method
driver = initialize_driver()

In [5]:
import logging
logging.basicConfig(filename='logs.log', level=logging.DEBUG, 
                    format='%(asctime)s %(levelname)s %(name)s %(message)s')
logger=logging.getLogger(__name__)

In [6]:
# DEFINE OUR STATIC KEYWORDS

import json
from itertools import chain


with open('Keywords.json', 'r') as f:
    data = json.load(f)
KEYWORDS=list(chain(*data.values()))
SCRAPED_URLS=[]

In [7]:
# Getting Youtube channels Names :
driver.get("https://eg.youtubers.me/senegal/news-politics/top-50-youtube-channels-in-senegal/en")
time.sleep(1)
elements = driver.find_elements(by=By.XPATH, value="//body[1]/div[4]/div[1]/div[1]/div[4]/div[1]/div[1]/div[1]/table[1]/tbody[1]/tr/td[2]/a[1]")
Youtube_Channels=[]
for e in elements:
    Youtube_Channels.append(e.text)
Youtube_Channels.append("La Maison Des Reporters")
print(Youtube_Channels)

['DAKARACTU TV HD', 'Walfadjri TV', 'Sen Tv Officiel [DMEDIA]', 'azactu', 'MakhPro boy kl', 'TMS TV Thierno Moule Sow', 'dakar instant', 'Thiesinfo.com', 'guediawayeinfos TV', 'KOLDA TV HD', 'Equivox TV HD', 'Koolo Hinde TV', 'Buzz Senegal Officiel', "L'Oeil Hibou TV", 'Touba infos', 'Thiey dakar', 'JTv / Senegal', 'LOUGA TV OFFICIEL', 'wiktivi - Mauritanie', 'baolmediasfm', 'La Vie Senegalaise', 'fecbeut tv', 'Jotay Net', 'TELE DAKAR', 'Sunuinfos TV HD', 'DAKAR 24', 'RDV TV', 'Mourides Tv HD', 'BTV (Banlieue TV Sénégal)', 'Keurdjeggdji TV', 'SenePlus TV', 'seneviral', '3M VISION', 'PARCELLES365', 'DoudouTV', 'Djoloff 24', 'MAGAL-TV EN DIRECT', 'Dalu Ndakaaru', 'KEPPAR TV', 'Tuning Studios', 'Diamiya TV', 'SUNUTUBE', "L' Oeil du Senegal", 'OUAKAM INFO', 'SENEGAL INTERNATION24', 'MEDIAS DAKAR TV', 'Ousdarou', 'SENEGAL-NEWS.NET', 'DreamProd', 'GALSEN 24', 'La Maison Des Reporters']


In [8]:
# A function to 
def get_number_of_videos(driver):
    titles=driver.find_elements(by=By.XPATH, value="//h3[@class='title-and-badge style-scope ytd-video-renderer']")
    return len(titles)

In [9]:
# function to add to JSON
def write_json(url,upload_date, filename='History.json'):
    #date=get_upload_date(url).strftime("%Y-%m-%d")
    now = datetime.now().strftime("%Y-%m-%d") # current date and time
    # get video id from url:
    video_id=url.split("=")[1]
    new_data={'id':video_id,'visiteDate':now,'uploadDate':upload_date}
    with open(filename,'r+') as file:
          # First we load existing data into a dict.
        file_data = json.load(file)
        # Join new_data with file_data inside emp_details
        file_data["scraped_urls"].append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, indent = 4)
        file.close()

In [10]:
import numpy as np
  
# function to get unique values
def unique(list1):
    x = np.array(list1)
    return list(np.unique(x))

In [11]:
# Define a function to search the item
def search_id(id,filename='History.json'):
    with open(filename,'r+') as file:
        file_data = json.load(file)
            # Join new_data with file_data inside emp_details
        items=file_data["scraped_urls"]
    for item in items:
        if id == item['id']:
            return item
    return None

In [12]:
# A function to perform keyword search on a specific channel, this function takes (channel name (string) and search query (keyword))
## This returns a url to the result page containing videos with specific keyword
def perform_keyword_search_on_channel(channel_name,search_query):
    query="https://www.youtube.com/results?search_query="+channel_name.replace(" ", "+")
    driver.get(query)
    time.sleep(1.5)
    link_to_channel =driver.find_elements(By.CSS_SELECTOR,value="#main-link")[0].get_attribute("href")
    search_query_link=link_to_channel+"/search?query="+search_query.replace(" ","%20")
    return search_query_link


In [14]:
# A function that gets all videos links in a result page, it takes two parameters: Search_query_link (the url to the result page) and the max number of videos the scrape (since we have infinity scroll)
## It returns the urls list
def get_videos_urls(search_query_link,max_videos):
    try:
        driver.get(search_query_link)
        
        if 'consent.youtube.com' in driver.current_url: # THIS IS THE CHECK
            driver.find_element_by_xpath('//button[@class="VfPpkd-LgbsSe VfPpkd-LgbsSe-OWXEXe-k8QpJ VfPpkd-LgbsSe-OWXEXe-dgl2Hf nCP5yc AjY5Oe DuMIQc LQeN7 IIdkle"]').click() # THIS ACCEPTS THE COOKIE CONSENT FORM
        
        while True:
            scroll_height = 40000
            document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
            driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
            time.sleep(1.5)
            document_height_after = driver.execute_script("return document.documentElement.scrollHeight")
            links=driver.find_elements(By.XPATH,value="//body/ytd-app[1]/div[1]/ytd-page-manager[1]/ytd-browse[1]/ytd-two-column-browse-results-renderer[1]/div[1]/ytd-section-list-renderer[1]/div[2]/ytd-item-section-renderer/div[3]/ytd-video-renderer[1]/div[1]/div[1]/div[1]/div[1]/h3[1]/a[1]")
            if document_height_after == document_height_before or len(links)>=max_videos:
                urls=[url.get_attribute("href") for url in links]
                return urls
    except IOError as e:
        logging.error('Error occurred ' + str(e))
        
## The number of returned urls will be not equal to specified max_videos number , but will be very close to it

In [16]:
def get_upload_date(url):
    driver.get(url)
    # Waits 20 seconds for the "date" the appear on the webpage
    date = WebDriverWait(driver, 3).until(
        EC.visibility_of_element_located(
            (By.CSS_SELECTOR, "#info-strings yt-formatted-string")
        )
    )

    return datetime.strptime(date.text, '%b %d, %Y')

In [ ]:
#get_upload_date('https://www.youtube.com/watch?v=Pr0tC-6hh44&ab_channel=JieJenn').strftime("%Y-%m-%d")

In [17]:
def update_json(video_id,filename="History.json"):
    now = datetime.now().strftime("%Y-%m-%d") # current date and time
    with open(filename,'r+') as file:
          # First we load existing data into a dict.
        file_data = json.load(file)
            # Join new_data with file_data inside emp_details
        items=file_data["scraped_urls"]
        for item in items:
            if video_id == item['id']:
                item["visiteDate"]=now
                break
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, indent = 4)
        file.close()

    

In [18]:
def filter_by_comments(url,keywords=KEYWORDS):
    # get comments of the video
     # get video id from url:
    video_id=url.split("=")[1]
    #output=f'/data/notebooks/DATA-ISBAINE/Youtube/Temp data/{video_id}.json'
    #!youtube-comment-downloader --youtubeid {video_id} --output '{output}'
    ydl_opts = {

            'noplaylist': True,
            'continue_dl': True,
            'outtmpl': f'/data/notebooks/DATA-ISBAINE/Youtube/Temp data/{video_id}',
            'geobypass':True,
            'ffmpeg_location':'/usr/bin/ffmpeg',
            "skip_download":True,
            'writeinfojson':True,
            'getcomments':True,

    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
       
        error_code = ydl.download(url)
    if error_code:
        print('Some videos failed to download')

    with open(f'/data/notebooks/DATA-ISBAINE/Youtube/Temp data/{video_id}.info.json', 'r') as metadata:
        data = json.load(metadata)
    # Get comments list and create a dataframe associated to it
    df=pd.DataFrame(data['comments'])
    upload_date=data["upload_date"]
    try:
        os.remove(f'/data/notebooks/DATA-ISBAINE/Youtube/Temp data/{video_id}.info.json')
    except IOError as e:
        logging.error('Error occurred ' + str(e))
    if len(df)>0:
        for keyword in keywords:
            df[keyword] = df['text'].apply(lambda _str: _str.lower().count(keyword))
        df['sum'] = df.sum(axis=1)
        count_keywords = sum(df['sum'].tolist())
    else:
        count_keywords=0
    return len(df),count_keywords,upload_date
     
          
    #os.system("source /data/venv/bin/activate")
    #os.system(f'youtube-comment-downloader --youtubeid {video_id} --output /data/notebooks/DATA-ISBAINE/Youtube/Temp data/{video_id}.json')

In [19]:
def add_to_list(video_url,desination="./To_Scrape.txt"):
    # Open the file in append & read mode ('a+')
    try:
        with open("./To_Scrape.txt", "a+") as file_object:
            file_object.seek(0)
            # If file is not empty then append '\n'
            list = file_object.readlines()
            found = False
            for line in list:
                if video_url in line:
                    found = True
            if not found:
                if len(list) > 0 :
                    file_object.write("\n")
                # Append text at the end of file
                file_object.write(video_url)
                # Close the file"""
 
            """text = file_object.read()

            with open("./To_Scrape.txt", "a+") as fobj:
                if not text.endswith('\n'):
                    fobj.write('\n')
                fobj.write(video_url)"""
            file_object.close()
            return True
    except:
        print("there was an error while saving the video url")
        return False

In [20]:
def filter_video(video_url,threshold):
    # get video id from url:
    video_id=video_url.split("=")[1]
    # check if the video have been scraped already: (if it's ID exists in the log file):
    already_scraped=search_id(video_id)
    if already_scraped!=None:
        number_of_comments,count,upload_date=filter_by_comments(video_url)
        delay=(datetime.strptime(datetime.now().strftime("%Y-%m-%d"),"%Y-%m-%d")- datetime.strptime(upload_date,"%Y%m%d")).days
        if delay<100:
            #number_of_comments,count=filter_by_comments(video_url)
            if count>threshold:
                # add to do list 
                add_to_list(video_url)
                update_json(video_id)
                
            
    else:
        number_of_comments,count,upload_date=filter_by_comments(video_url)
        if count>threshold:
                # add to do list 
            add_to_list(video_url)
            write_json(video_url,datetime.strptime(upload_date,"%Y%m%d").strftime("%Y-%m-%d"))
            

In [ ]:
#datetime.strptime("20200315","%Y%m%d").strftime("%Y-%m-%d")

In [21]:
def get_videos_by_channel(channel_name,max_videos):
    urls=[]
    for i in tqdm(range(1)):
        link=perform_keyword_search_on_channel(channel_name,KEYWORDS[i])
        urls.extend(get_videos_urls(link,max_videos))
        SCRAPED_URLS.extend(get_videos_urls(link,max_videos))
    return urls

In [ ]:
"""def main():
    MAX_VIDEOS=5000
    for i in tqdm(range(len(Youtube_Channels))):
        urls=get_videos_by_channel(Youtube_Channels[i],MAX_VIDEOS)
        print(len(urls))
    return urls
urls=main()"""

In [22]:
def get_data():
    MAX_VIDEOS=3000
    for i in tqdm(range(1)):
        try:
            urls=get_videos_by_channel(Youtube_Channels[i],MAX_VIDEOS)
            print(len(urls))
        except IOError as e:
            logging.error('Error occurred ' + str(e))
            pass
        


In [23]:
def main():
    get_data()
    links=unique(SCRAPED_URLS)
    for i in tqdm(range(len(links))):
        filter_video(links[i],10)

In [24]:
main()

100%|██████████| 1/1 [00:44<00:00, 44.06s/it]


329


  0%|          | 0/329 [00:00<?, ?it/s]DeprecationWarning: Support for Python version 3.6 has been deprecated and will break in future versions of yt-dlp! Please update to Python 3.7 or above


[youtube] -0OcflFWFAQ: Downloading webpage
[youtube] -0OcflFWFAQ: Downloading android player API JSON
[youtube] -0OcflFWFAQ: Downloading MPD manifest
[youtube] -0OcflFWFAQ: Downloading MPD manifest
[youtube] Downloading comment section API JSON
[youtube] Downloading ~1 comments
[youtube] Sorting comments by newest first
[youtube] Downloading comment API JSON page 1 (0/1)
[youtube] Extracted 1 comments
[info] -0OcflFWFAQ: Downloading 1 format(s): 243+251
[info] Writing video metadata as JSON to: /data/notebooks/DATA-ISBAINE/Youtube/Temp data/-0OcflFWFAQ.info.json


  0%|          | 1/329 [00:02<11:07,  2.04s/it]DeprecationWarning: Support for Python version 3.6 has been deprecated and will break in future versions of yt-dlp! Please update to Python 3.7 or above


...

 99%|█████████▉| 327/329 [06:07<00:02,  1.16s/it]DeprecationWarning: Support for Python version 3.6 has been deprecated and will break in future versions of yt-dlp! Please update to Python 3.7 or above


[youtube] zhFJwxpejDU: Downloading webpage
[youtube] zhFJwxpejDU: Downloading android player API JSON
[youtube] zhFJwxpejDU: Downloading MPD manifest
[youtube] zhFJwxpejDU: Downloading MPD manifest
[youtube] Downloading comment section API JSON
[youtube] Downloading ~4 comments
[youtube] Sorting comments by newest first
[youtube] Downloading comment API JSON page 1 (0/4)
[youtube] Extracted 4 comments
[info] zhFJwxpejDU: Downloading 1 format(s): 248+251
[info] Writing video metadata as JSON to: /data/notebooks/DATA-ISBAINE/Youtube/Temp data/zhFJwxpejDU.info.json


100%|█████████▉| 328/329 [06:08<00:01,  1.22s/it]DeprecationWarning: Support for Python version 3.6 has been deprecated and will break in future versions of yt-dlp! Please update to Python 3.7 or above


[youtube] zy7bETitQ4k: Downloading webpage
[youtube] zy7bETitQ4k: Downloading android player API JSON
[youtube] Downloading comment section API JSON
[youtube] Sorting comments by newest first
[youtube] Downloading comment API JSON page 1 (0/0)
[youtube] Extracted 0 comments
[info] zy7bETitQ4k: Downloading 1 format(s): 248+251
[info] Writing video metadata as JSON to: /data/notebooks/DATA-ISBAINE/Youtube/Temp data/zy7bETitQ4k.info.json


100%|██████████| 329/329 [06:09<00:00,  1.12s/it]


In [ ]:
"""def search_keywords(keywords):
    driver.get("https://www.youtube.com/?persist_gl=1&gl=SN")
    try:
        search_input = driver.find_elements(by=By.XPATH, value="//body[1]/ytd-app[1]/div[1]/div[1]/ytd-masthead[1]/div[3]/div[2]/ytd-searchbox[1]/form[1]/div[1]/div[1]/input[1]")
        # Finding the search field by id
        # Sending input text to search field
        search_input.send_keys(keywords)

        # Pressing enter to search input text
        search_input.send_keys(Keys.ENTER)
        time.sleep(1)
        
        while True:
        scroll_height = 2000
        document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
        time.sleep(1.5)
        document_height_after = driver.execute_script("return document.documentElement.scrollHeight")
        if document_height_after == document_height_before or :
            break
        #//body[1]/ytd-app[1]/div[1]/ytd-page-manager[1]/ytd-search[1]/div[1]/ytd-two-column-search-results-renderer[1]/div[1]/ytd-section-list-renderer[1]/div[2]/ytd-item-section-renderer[3]/div[3]/ytd-video-renderer[20]/div[1]/div[1]/div[1]/div[1]/h3[1]/a[1]
        #title-and-badge style-scope ytd-video-renderer
    except:
        print("Error")
#//body/ytd-app[1]/div[1]/ytd-page-manager[1]/ytd-browse[1]/ytd-two-column-browse-results-renderer[1]/div[1]/ytd-section-list-renderer[1]/div[2]/ytd-item-section-renderer[14]/div[3]/ytd-video-renderer[1]/div[1]/div[1]/div[1]/div[1]/h3[1]/a[1]
#//body/ytd-app[1]/div[1]/ytd-page-manager[1]/ytd-browse[1]/ytd-two-column-browse-results-renderer[1]/div[1]/ytd-section-list-renderer[1]/div[2]/ytd-item-section-renderer[15]/div[3]/ytd-video-renderer[1]/div[1]/div[1]/div[1]/div[1]/h3[1]/a[1]"""